# Deep Agent - Multi-Agent Deep Finance Researcher

Advanced financial research system using LangChain's DeepAgent with context isolation and strategic delegation.

## Overview

This notebook demonstrates:
- **DeepAgent Framework**: LangChain's advanced agent architecture
- **Context Isolation**: Sub-agents with isolated contexts for focused research
- **Strategic Delegation**: Orchestrator coordinates multiple specialist sub-agents
- **RAG + Live Data**: Hybrid search on SEC filings + Yahoo Finance integration
- **File-Based Memory**: Persistent research artifacts with secure sandbox (virtual_mode=True)

**Key Features:**
1. Orchestrator breaks down complex queries into research tasks
2. Sub-agents conduct focused research with isolated context
3. Think tool for strategic reflection and decision-making
4. Final report synthesis with consolidated citations
5. SQLite checkpointer for conversation memory
6. FilesystemBackend with virtual_mode for secure file operations

## Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import uuid
import sqlite3
from datetime import datetime
from IPython.display import Image, display

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from deepagents.backends.utils import file_data_to_string
from langgraph.checkpoint.sqlite import SqliteSaver

from scripts.rag_tools import hybrid_search, live_finance_researcher, think_tool
from scripts.deep_prompts import DEEP_RESEARCHER_INSTRUCTIONS, DEEP_ORCHESTRATOR_INSTRUCTIONS

## Initialize Memory and File Backend

In [4]:
# Secure filesystem backend for research outputs
RESEARCH_OUTPUT_DIR = os.path.join(os.getcwd(),"research_outputs")

In [5]:
def get_research_backend(user_id, thread_id):

    USER_OUTPUT_DIR = os.path.join(RESEARCH_OUTPUT_DIR, user_id, thread_id)
    
    os.makedirs(USER_OUTPUT_DIR, exist_ok=True)

    print(f"Writing research files to: {USER_OUTPUT_DIR}")

    # Create filesystem backend with virtual_mode=True for security
    backend = FilesystemBackend(
        root_dir=USER_OUTPUT_DIR,
        virtual_mode=True  # Prevents agent from accessing files outside sandbox
    )

    return backend

## Create Research Sub-Agent

In [6]:
# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Create research sub-agent with isolated context
research_sub_agent = {
    "name": "financial-research-agent",
    "description": "Delegate financial research to this sub-agent. Give it one specific research task at a time.",
    "system_prompt": DEEP_RESEARCHER_INSTRUCTIONS.format(date=current_date),
    "tools": [hybrid_search, live_finance_researcher, think_tool],
}

## Initialize LLM and Create DeepAgent

In [7]:
# Initialize model
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

# Tools for the main agent (orchestrator level)
tools = [hybrid_search, live_finance_researcher, think_tool]

In [8]:
def get_deep_agent(user_id, thread_id):

    # SQLite checkpointer for agent memory
    conn = sqlite3.connect('data/deep_agent_finance_researcher.db', check_same_thread=False)
    checkpointer = SqliteSaver(conn=conn)

    backend = get_research_backend(user_id, thread_id)

    # Create the deep agent with memory and secure file backend
    agent = create_deep_agent(
        model=model,
        tools=tools,
        system_prompt=DEEP_ORCHESTRATOR_INSTRUCTIONS,
        subagents=[research_sub_agent],
        checkpointer=checkpointer,  # SQLite memory
        backend=backend,  # Secure filesystem with virtual_mode=True
    )

    return agent

    # Visualize the agent graph
    agent

## Example 1: Simple Financial Query

In [9]:
from scripts.agent_utils import stream_agent_response

query = "What was Amazon's revenue in Q1 2024?"
user_id = "kgptalkie"
thread_id="session1"

agent = get_deep_agent(user_id, thread_id)
stream_agent_response(agent, query, thread_id)

Writing research files to: d:\Courses\Udemy\Multi Agent Deep RAG\research_outputs\kgptalkie\session1
Amazon's total revenue for the first quarter of 2024 was **$143.31 billion** ($143,313 million), representing a **13% year-over-year increase** from the $127.36 billion reported in Q1 2023 [1].

### Revenue by Segment
*   **North America:** $86.34 billion (up 12% YoY)
*   **International:** $31.94 billion (up 10% YoY)
*   **AWS (Amazon Web Services):** $25.04 billion (up 17% YoY)

### Sources
[1] Source file: Amazon Q1 2024 Form 10-Q, page 25.

In [10]:
# Files are already displayed by stream_deep_agent helper
# The final report is printed automatically if available

In [11]:
query = "Compare Apple and Amazon's 2024 revenue and profitability. Present full and detailed report."
thread_id="session2"

agent = get_deep_agent(user_id, thread_id)
stream_agent_response(agent, query, thread_id)

Writing research files to: d:\Courses\Udemy\Multi Agent Deep RAG\research_outputs\kgptalkie\session2

  Tool Called: ls
   Args: {'path': '/'}


  Tool Result (length: 2 chars)

# Financial Comparison: Apple Inc. vs. Amazon.com, Inc. (2024)

## 1. Introduction
This report provides a detailed comparison of the revenue and profitability of Apple Inc. and Amazon.com, Inc. for the 2024 period. The analysis utilizes Apple's full fiscal year 2024 results (ended September 28, 2024) and Amazon's results for the first nine months of the 2024 calendar year (January through September 2024).

## 2. Apple 2024 Financial Overview (Fiscal Year)
Apple's fiscal year 2024 was marked by a return to revenue growth and record-shattering performance in its Services segment.

### Key Financial Metrics (FY 2024)
*   **Total Revenue:** $391,035 million (+2.0% YoY) [1]
*   **Gross Margin:** $180,683 million (46.2% margin), an improvement from 44.1% in 2023 [1]
*   **Operating Income:** $123,216 million (31.5% m

In [12]:
# Files are already displayed by stream_deep_agent helper
# The final report is printed automatically if available

In [ ]:
query = "Compare Apple and Google's 2024 revenue and profitability"
thread_id="session3"

agent = get_deep_agent(user_id, thread_id)
stream_agent_response(agent, query, thread_id)

Writing research files to: d:\Courses\Udemy\Multi Agent Deep RAG\research_outputs\kgptalkie\session3

  Tool Called: write_todos
   Args: {'todos': [{'status': 'in_progress', 'content': 'Save research request to /research_request.md'}, {'status': 'pending', 'content': "Research Apple's 2024 revenue and profitability using a sub-agent"}, {'status': 'pending', 'content': "Research Google (Alphabet)'s 2024 revenue and profitability using a sub-agent"}, {'status': 'pending', 'content': 'Synthesize findings and consolidate citations'}, {'status': 'pending', 'content': 'Write final report to /final_report.md'}, {'status': 'pending', 'content': 'Verify report against /research_request.md'}]}


  Tool Called: write_file
   Args: {'file_path': '/research_request.md', 'content': "Compare Apple and Google's 2024 revenue and profitability"}


  Tool Result (length: 565 chars)


  Tool Result (length: 33 chars)


  Tool Called: write_todos
   Args: {'todos': [{'status': 'completed', 'content': 'Sav

## Example 4: Historical + Live Data

In [ ]:
query = "Research Meta's 2024 annual financial performance from SEC filings and compare it with current stock performance"
thread_id="session4"

agent = get_deep_agent(thread_id)
stream_agent_response(agent, query, thread_id)